In [4]:
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import pandas as pd
import os
from tqdm import tqdm

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
dataset_filtered = load_from_disk("/home/jisukim/LLMscience/openbook/dataset/stem")

In [3]:
dataset_filtered['text'][0]

Dataset({
    features: ['title', 'section', 'text'],
    num_rows: 2101279
})

In [24]:
df = pd.DataFrame()

In [25]:
df['title'] = dataset_filtered['title']

In [27]:
df['section'] = dataset_filtered['section']

In [28]:
df['text'] = dataset_filtered['text']

In [30]:
df.to_csv('/home/jisukim/LLMscience/openbook/dataset/stem_wiki.csv', index=True)

In [3]:
df = pd.read_csv('/home/jisukim/LLMscience/openbook/dataset/stem_wiki.csv')

In [ ]:
texts = []

for i in tqdm(range(len(df))):
    tot_text = "[CLS]" + str(df.iloc[i]['title']) + "[SEP]" + str(df.iloc[i]['section']) + "[SEP]" + str(df.iloc[i]['text'])
    texts.append(tot_text)

In [ ]:
df['ctx'] = texts
df['index'] = df.index

In [ ]:
df[['index', 'ctx']].to_parquet('/home/jisukim/LLMscience/openbook/dataset/stem_wiki_only_ctx.parquet')

### Maxlength 체크

In [5]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L12-v2')

In [7]:
text_length = []

for i in tqdm(range(len(df))):
    text_length.append(tokenizer.tokenize("[CLS]" + str(df.iloc[i]['title']) + "[SEP]" + str(df.iloc[i]['section']) + "[SEP]" + str(df.iloc[i]['text'])))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2101279/2101279 [33:15<00:00, 1052.86it/s]


In [10]:
txt_len = []

for i in tqdm(range(len(text_length))):
    txt_len.append(len(text_length[i]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2101279/2101279 [00:01<00:00, 1826397.76it/s]


In [17]:
import numpy as np

In [18]:
q1 = np.percentile(txt_len, 25)  # 1사분위수
median = np.percentile(txt_len, 50)  # 중앙값 (2사분위수)
q3 = np.percentile(txt_len, 75)

In [30]:
q4 = np.percentile(txt_len, 95)